First set the key you received from Microsoft Azure in order to use the Bing API

In [ ]:
key = os.environ.get('AZURE_SEARCH_KEY', 'YOUR KEY')

The free version of Azure lets returns up to 150 search results. Here is how you search and download:

Download all images and put them in different folders:

In [ ]:
bear_types = 'grizzly','black','teddy'
path = Path('bears')

In [ ]:
if not path.exists():
    path.mkdir()
    for o in bear_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o} bear')
        download_images(dest, urls=results.attrgot('contentUrl'))

Check whether the folder actually has the images

In [ ]:
fns = get_image_files(path)
fns

Check for corrupted images and unlink those images

In [ ]:
failed = verify_images(fns)
failed.map(Path.unlink);

To generate a data loader object in order to feed the images to the network for training we first generate a datablock.
blocks: define independent (the thing we make predictions from = images) and 
dependent variable (the category = type of bear)
get_items: underlying items = list of all image files
splitter: split data in train and valid "randomly" with 80% for train
gety: label the dependet variable y with the parent folder name (type of bear)
item_tfms (item transforms run on each item, here we use resize, which has 
different modes of resizing (crop, patch, squish, ...); 
best practice is to randomize the image patch we crop out and feed a different one in each epoch
item_tfms=RandomResizedCrop(128, min_scale=0.3); this is a form of data
augmentation; other default augmentations can be introduced on whole batches 
after resize on each item:
bears = bears.new(item_tfms=Resize(128), batch_tfms=aug_transforms(mult=2)))
to change the item_tfms fct in the datablock object bear use
bears = bears.new(item_tfms=Resize(128, ResizeMethod.Squish))

In [ ]:
bears = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

Generate dataloader from datablock and display one batch to check

In [ ]:
dls = bears.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

If we wanted to change something in the DataBlock, we can use the following

In [ ]:
bears = bears.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())

dls = bears.dataloaders(path)

Create learner and fine tune for the classification at hand

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In order to judge the classification success we can look at the missclassification in teh form of a confusion matrix...

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

and also check the top loss images

In [ ]:
interp.plot_top_losses(5, nrows=1)

In order to clean the data we can use a fastai widget called ImageClassifierCleaner

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
And delete the corresponding ideces: 

In [ ]:
#hide
for idx in cleaner.delete(): cleaner.fns[idx].unlink()
# for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)